In [4]:
import math
import torch
from diffusion_edf.dist import IgSO3Dist, isotropic_gaussian_so3, isotropic_gaussian_so3_small_angle, isotropic_gaussian_so3_lie_deriv, isotropic_gaussian_so3_score
from diffusion_edf import transforms
import plotly.graph_objs as go

In [5]:
dist = IgSO3Dist()

In [6]:
dtype = torch.float64
device = 'cpu'

eps = 1e-8
q = dist.sample(eps = eps, N=3, dtype=dtype, device=device)
# theta = 0.4
# # q = torch.tensor([[math.cos(theta/2), math.sin(theta/2), 0., 0.]], dtype=dtype, device=device)
# randomaxis = torch.randn(1,3, dtype=dtype, device=device)
# randomaxis = randomaxis / randomaxis.norm(dim=-1, keepdim=True) * math.sin(theta/2)
# q = torch.cat([torch.ones_like(randomaxis[...,:1]) * math.cos(theta/2), randomaxis], dim=-1)

log_prob = torch.log(isotropic_gaussian_so3(q, eps))
score = isotropic_gaussian_so3_score(q, eps)
print(f"Anaylitic score: {score}")

dt = 0.0000001

log_prob_rot = []
for i in range(3):
    qrot = transforms.quaternion_multiply(q,transforms.axis_angle_to_quaternion(dt * torch.eye(3, device=q.device, dtype=q.dtype)[i]))
    log_prob_rot.append(torch.log(isotropic_gaussian_so3(qrot, eps)))

log_prob_rot = torch.stack(log_prob_rot, dim=-1)
print(f"Numerical score: {(log_prob_rot - log_prob[...,None]) / dt}")
print(f"Allclose: {torch.allclose((log_prob_rot - log_prob[...,None]) / dt, score, atol=0, rtol=1e-2)}")

Anaylitic score: tensor([[12126.3133, -2197.8743,   718.1253],
        [ 1764.0940, -8634.6239,  5091.8354],
        [-9647.5560, -2980.0231,  2135.3295]], dtype=torch.float64)
Numerical score: tensor([[12123.8419, -2200.4179,   715.6355],
        [ 1761.6319, -8637.1479,  5089.4551],
        [-9650.0451, -2982.4507,  2132.9554]], dtype=torch.float64)
Allclose: True
